# Sudoku Solver

## Udacity Artificial Intelligence Nanodegree program



### Part 1 - Generate the puzzle

In this section, we will use BeautifulSoup to web scrape and return puzzles from a Sudoku website.

In [1]:
from bs4 import BeautifulSoup
import requests
import random

In [2]:
def make_soup(level):
    """ This function takes the puzzle difficulty level as an input and return a HTML soup """
    levels = {'Easy':1, 'Medium':2, 'Hard':3, 'Evil':4}
    url = "http://view.websudoku.com/?level={}".format(levels[level])
    response = requests.get(url)
    html = response.text
    return BeautifulSoup(html,'html.parser') 

In [3]:
def build_puzzle_string(soup):
    """ This function takes a HTML soup as an input and return a puzzle string """
    
    grid = soup.find(id="puzzle_grid") 
    puzzle_string = ''
    
    for elements in grid.find_all('td'):
        for element in elements:
            if element.get('value'):
                puzzle_string += element.get('value')
            else:
                puzzle_string += '.'
    return puzzle_string

In [4]:
def get_puzzle(level):
    """ This function takes the puzzle difficulty level as an input and return the puzzle string """
    soup = make_soup(level)
    return build_puzzle_string(soup)

### Part 2 - Solving the puzzle

### Definitions:

**Boxes, Units and Peers**

* The individual squares at the intersection of rows and columns will be called boxes. These boxes will have labels 'A1', 'A2', ..., 'I9'.

* The complete rows, columns, and 3x3 squares, will be called units. Thus, each unit is a set of 9 boxes, and there are 27 units in total.

* For a particular box (such as 'A1'), its peers will be all other boxes that belong to a common unit (namely, those that belong to the same row, column, or 3x3 square).

In [5]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [6]:
def cross(a,b):
    return [s+t for s in a for t in b]

In [7]:
row_units = [cross(r, cols) for r in rows]
#print(row_units)

In [8]:
column_units = [cross(rows, c) for c in cols]
#print(column_units)

In [9]:
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
#print(square_units)

In [10]:
unitlist = row_units + column_units + square_units
#print(unitlist)

In [11]:
boxes = (cross(rows,cols))

In [12]:
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)

In [13]:
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [14]:
def replace_dots(grid_string):
    
    assert len(grid_string) == 81, "Invalid number of elements in string."
    
    return ['123456789' if x=='.' else x for x in grid_string] 

In [15]:
def grid_values(grid_string):
    
    assert len(grid_string) == 81, "Invalid number of elements in string."
       
    return dict(zip(boxes, grid_string))   

In [16]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [17]:
def eliminate(values):
    
    for key, value in values.items():
        if len(value) == 1:
            for peer in peers[key]:
                values[peer] = values[peer].replace(value,'') 
    return values   
    

In [18]:
def only_choice(values):
    
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [19]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [20]:
def depth_first_search(values):
    # Reduce the puzzle 
    values = reduce_puzzle(values)
    
    if values is False:
        return False ## Failed earlier
    
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = depth_first_search(new_sudoku)
        if attempt:
            return attempt

### Testing

This section will randomly select a puzzle difficulty level, get the puzzle from the internet, solve the puzzle and print the results.

In [33]:
difficulty_list = ["Easy", "Medium","Hard", "Evil"]  

selected_difficulty = random.choice(difficulty_list)
puzzle = get_puzzle(selected_difficulty)

print("\nHere is the " + selected_difficulty + " Level puzzle:")
display(grid_values(puzzle))

print("\nHere is the solution for the " + selected_difficulty + " Level puzzle:")
display(depth_first_search(grid_values(replace_dots(puzzle))))


Here is the Medium Level puzzle:
. . . |. 6 . |. . 9 
. 9 . |4 . . |. 7 2 
. . 1 |. 5 9 |8 3 . 
------+------+------
. 6 9 |. 8 . |7 . . 
. . . |. 3 . |. . . 
. . 3 |. 4 . |9 2 . 
------+------+------
. 5 4 |3 2 . |1 . . 
1 7 . |. . 5 |. 4 . 
8 . . |. 7 . |. . . 

Here is the solution for the Medium Level puzzle:
3 4 7 |8 6 2 |5 1 9 
5 9 8 |4 1 3 |6 7 2 
6 2 1 |7 5 9 |8 3 4 
------+------+------
4 6 9 |2 8 1 |7 5 3 
2 1 5 |9 3 7 |4 8 6 
7 8 3 |5 4 6 |9 2 1 
------+------+------
9 5 4 |3 2 8 |1 6 7 
1 7 2 |6 9 5 |3 4 8 
8 3 6 |1 7 4 |2 9 5 
